In [ ]:
import glob
import os
from pathlib import Path

import yaml
from deepdiff import DeepDiff
from utils import (
    amputees,
    channel_names,
    intact,
    participants,
    recordings,
    targets,
    test_recordings,
)

In [ ]:
paper_plots_path = Path("/Users/jorisg/Desktop/upper_limb/paper_figures_revision")
base_dir = Path().resolve().parent

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(-10, 10, 500)
softplus = np.log1p(np.exp(x))  # log(1 + exp(x))

plt.figure(figsize=(8, 5))
plt.plot(x, softplus, label="softplus(x)", linewidth=2)
plt.title("Softplus Activation Function")
plt.xlabel("x")
plt.ylabel("softplus(x)")
plt.grid(True)
plt.legend()
plt.show()


-> P7_453 is Right, all others are Left


In [ ]:
base_config_path = base_dir / "data/<person_ID>/configs/modular_initialization.yaml"
with open(base_config_path, "r") as f:
    base_config_template = yaml.safe_load(f)

base_config_template["parameters"]["activation_model"] = {
    "parameters": {
        "enforce_positive_weights": {"values": [False, True]},
        "model_type": {"value": "MatMul"},
        "n_freeze_epochs": {"value": 0},
        "init": {"values": ["default", "xavier_normal"]},
    }
}
base_config_template["parameters"]["wandb_project"]["value"] = (
    "model_comparison_experiments"
)
base_config_template["parameters"]["learning_rate"]["values"] = [
    0.1,
    0.01,
    0.001,
    0.0001,
]
base_config_template["parameters"]["learning_rate"].pop("value", None)

base_config_template["parameters"]["dummy_repetition"]["value"] = 1
base_config_template["parameters"]["dummy_repetition"].pop("values", None)

for participant in participants:
    participant_config_path = (
        base_dir / f"data/{participant}/configs/modular_MatMul.yaml"
    )

    base_config = base_config_template.copy()
    base_config["name"] = participant

    if participant == "P7_453":
        base_config["parameters"]["targets"]["value"] = [
            ["Right", joint]
            for _, joint in base_config["parameters"]["targets"]["value"]
        ]
    else:
        base_config["parameters"]["targets"]["value"] = [
            ["Left", joint]
            for _, joint in base_config["parameters"]["targets"]["value"]
        ]
    with open(participant_config_path, "w") as f:
        yaml.dump(base_config, f)
    print(f"Copied config to {participant_config_path}")


In [ ]:
import subprocess

for participant in participants:
    hand = "Right" if participant == "P7_453" else "Left"
    print(f"Running training for {participant} with intact hand {hand}")

    # Run the command using subprocess
    subprocess.run(
        [
            "python",
            base_dir / "s4_train.py",
            "--person_dir",
            participant,
            "--intact_hand",
            hand,
            "--config_name",
            "modular_MatMul",
            "-hs",
        ],
        cwd=base_dir,
    )


In [ ]:
import pandas as pd

import wandb

api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("MIT_biomech/initialization_experiments")

summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"summary": summary_list, "config": config_list, "name": name_list}
)

# runs_df.to_csv("project.csv")

In [ ]:
runs_df["total_val_loss"] = runs_df["summary"].apply(
    lambda x: x.get("total_val_loss", None)
)
# total_test_loss
runs_df["total_test_loss"] = runs_df["summary"].apply(
    lambda x: x.get("total_test_loss", None)
)
runs_df["init"] = runs_df["config"].apply(
    lambda x: x.get("activation_model").get("init", None)
)

In [ ]:
runs_df.drop(columns=["summary", "config"], inplace=True)
runs_df.rename(columns={"name": "participant"}, inplace=True)


In [ ]:
runs_df["total_combined_loss"] = (
    runs_df["total_val_loss"] * 9 + runs_df["total_test_loss"] * 3
) / 12

In [ ]:
# group by participant and init, then get the mean and std of total_val_loss
grouped_df = (
    runs_df.groupby(["participant", "init"])
    .agg(
        mean_val_loss=("total_val_loss", "mean"),
        std_val_loss=("total_val_loss", "std"),
        mean_test_loss=("total_test_loss", "mean"),
        std_test_loss=("total_test_loss", "std"),
        mean_combined_loss=("total_combined_loss", "mean"),
        std_combined_loss=("total_combined_loss", "std"),
    )
    .reset_index()
)

In [ ]:
grouped_df_amputee = grouped_df[grouped_df["participant"].isin(amputees)].copy()
grouped_df_intact = grouped_df[grouped_df["participant"].isin(intact)].copy()

In [ ]:
# average over all participants
average_df_amputee = (
    grouped_df_amputee.groupby("init")
    .agg(
        mean_mean_val_loss=("mean_val_loss", "mean"),
        mean_std_val_loss=("std_val_loss", "mean"),
        mean_mean_test_loss=("mean_test_loss", "mean"),
        mean_std_test_loss=("std_test_loss", "mean"),
        mean_mean_combined_loss=("mean_combined_loss", "mean"),
        mean_std_combined_loss=("std_combined_loss", "mean"),
    )
    .reset_index()
)
average_df_intact = (
    grouped_df_intact.groupby("init")
    .agg(
        mean_mean_val_loss=("mean_val_loss", "mean"),
        mean_std_val_loss=("std_val_loss", "mean"),
        mean_mean_test_loss=("mean_test_loss", "mean"),
        mean_std_test_loss=("std_test_loss", "mean"),
        mean_mean_combined_loss=("mean_combined_loss", "mean"),
        mean_std_combined_loss=("std_combined_loss", "mean"),
    )
    .reset_index()
)

In [ ]:
average_df_intact.index = average_df_intact["init"]
average_df_amputee.index = average_df_amputee["init"]
average_df_intact.drop(columns=["init"], inplace=True)
average_df_amputee.drop(columns=["init"], inplace=True)

In [ ]:
# round to 3 decimal places
average_df_amputee = average_df_amputee.round(3)
average_df_intact = average_df_intact.round(3)

In [ ]:
# transpose the dataframes
average_df_amputee = average_df_amputee.T
average_df_intact = average_df_intact.T

In [ ]:
# Move the distinction between mean and std to the column names
# The new columns will be MultiIndex: (Initialization_method, Statistic)
new_df = pd.DataFrame(
    index=pd.MultiIndex.from_product(
        [["Combined", "New Movements", "Known Movements"], ["Intact", "Amputation"]],
        names=["Data Set", "Cohort"],
    ),
    columns=pd.MultiIndex.from_product(
        [average_df_amputee.columns, ["mean", "std"]],
        names=["Initialization_method", "Statistic"],
    ),
)
new_df


In [ ]:
for column in average_df_amputee.columns:
    new_df.loc[("Combined", "Amputation"), (column, "mean")] = average_df_amputee.loc[
        "mean_mean_combined_loss", column
    ]
    new_df.loc[("Combined", "Amputation"), (column, "std")] = average_df_amputee.loc[
        "mean_std_combined_loss", column
    ]
    new_df.loc[("New Movements", "Amputation"), (column, "mean")] = (
        average_df_amputee.loc["mean_mean_test_loss", column]
    )
    new_df.loc[("New Movements", "Amputation"), (column, "std")] = (
        average_df_amputee.loc["mean_std_test_loss", column]
    )
    new_df.loc[("Known Movements", "Amputation"), (column, "mean")] = (
        average_df_amputee.loc["mean_mean_val_loss", column]
    )
    new_df.loc[("Known Movements", "Amputation"), (column, "std")] = (
        average_df_amputee.loc["mean_std_val_loss", column]
    )
for column in average_df_intact.columns:
    new_df.loc[("Combined", "Intact"), (column, "mean")] = average_df_intact.loc[
        "mean_mean_combined_loss", column
    ]
    new_df.loc[("Combined", "Intact"), (column, "std")] = average_df_intact.loc[
        "mean_std_combined_loss", column
    ]
    new_df.loc[("New Movements", "Intact"), (column, "mean")] = average_df_intact.loc[
        "mean_mean_test_loss", column
    ]
    new_df.loc[("New Movements", "Intact"), (column, "std")] = average_df_intact.loc[
        "mean_std_test_loss", column
    ]
    new_df.loc[("Known Movements", "Intact"), (column, "mean")] = average_df_intact.loc[
        "mean_mean_val_loss", column
    ]
    new_df.loc[("Known Movements", "Intact"), (column, "std")] = average_df_intact.loc[
        "mean_std_val_loss", column
    ]

In [ ]:
# reorder Initialization_method: [default_all, default_0, default, kaiming_normal, xavier_normal, orthogonal]
new_df = new_df.reindex(
    columns=[
        ("default_all", "mean"),
        ("default_all", "std"),
        ("default_0", "mean"),
        ("default_0", "std"),
        ("default", "mean"),
        ("default", "std"),
        ("kaiming_normal", "mean"),
        ("kaiming_normal", "std"),
        ("xavier_normal", "mean"),
        ("xavier_normal", "std"),
        ("orthogonal", "mean"),
        ("orthogonal", "std"),
    ]
)


In [ ]:
new_df.to_csv(paper_plots_path / "initialization_experiment_results.csv")

In [ ]:
# replace this: [default_all, default_0, default, kaiming_normal, xavier_normal, orthogonal]
# with a multi_index:
# LeakyReLu Initialization: [Default, Default, Default, Kaiming Normal, Xavier Normal, Orthogonal]
# Sigmoid Initialization: [Default, Default, Xavier Normal, Xavier Normal, Xavier Normal, Xavier Normal]
# Bias Initialization: [Default, Zero, Zero, Zero, Zero, Zero]


In [ ]:
new_df